# Generation de musique avec des LSTM

- installer le package music21 via conda/pip
- verifier que les imports suivant passent bien

In [6]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import utils
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

- extrayons les notes de nos fichiers .mid (et sauvegardons le résultat, comme l'extraction peut prendre du temps)
- les fichiers .mid ne sont pas un fichier "audio" tel qu'un .mp3, mais davantage une partition, qui contiennent l'ensemble
des notes, melodies, rythme et instruments de la musique

In [1]:
"""
notes = []

for file in glob.glob("data/*.mid"):
    try:
        midi = converter.parse(file)
    except:
        print("unable to read ", file)
        continue
    print("Parsing %s" % file)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            print(element.pitch)
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            print('.'.join(str(n) for n in element.normalOrder))
            notes.append('.'.join(str(n) for n in element.normalOrder))
    
with open('notes', 'wb') as filepath:
    pickle.dump(notes, filepath)
"""

'\nnotes = []\n\nfor file in glob.glob("data/*.mid"):\n    try:\n        midi = converter.parse(file)\n    except:\n        print("unable to read ", file)\n        continue\n    print("Parsing %s" % file)\n\n    notes_to_parse = None\n\n    try: # file has instrument parts\n        s2 = instrument.partitionByInstrument(midi)\n        notes_to_parse = s2.parts[0].recurse() \n    except: # file has notes in a flat structure\n        notes_to_parse = midi.flat.notes\n\n    for element in notes_to_parse:\n        if isinstance(element, note.Note):\n            print(element.pitch)\n            notes.append(str(element.pitch))\n        elif isinstance(element, chord.Chord):\n            print(\'.\'.join(str(n) for n in element.normalOrder))\n            notes.append(\'.\'.join(str(n) for n in element.normalOrder))\n    \nwith open(\'notes\', \'wb\') as filepath:\n    pickle.dump(notes, filepath)\n'

In [21]:
def load_notes():
    notes = []

    with open('notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    return notes
notes = load_notes()

- qu'y a-t-il dans nos notes ?
    - format lettre+chiffre : code une note (ex : A5). La suite de note forme la mélodie de la musique.
    - format chiffre.chiffre.chiffre : code un accord (l'harmonie) : lorsque plusieurs notes sont superposées au même moment.

In [13]:
print(notes)

['D5', '5', 'D5', '3.5.9', '0.3.7', '10.2.5', '7.9.0', '7.10.2', '3.6.10', '5.7.0', '3.8', 'G4', 'B-4', '5.7.0', 'B-4', 'B-4', 'B-4', '10.11.1.4.6', '10.11.1.4.6', 'B-4', 'G#4', '10.0.3.5', 'C4', '5.7.10', 'D2', 'B4', 'E4', 'F#3', '8.0', '11.4', '8.1', '8.10.11', 'E-4', '5.11', '3.8', '4.7.10.0', '1.4.7', '9.10.1', '9.10.1.4', '4.9', '7', '0.3.5.8', '0.3.7', '5.8.0', '0.5', '3', '8.10', '5', '0.4', '7', '10.1.4', 'B-3', '8.0.3', '0.3.4.7', '5', '2.5.8', '8.0.2', '2.5.8.10', 'B-4', '0.1.2', '10.2.3', 'E-5', '7.10.0', 'D5', '10.0', '10.11.0', '5.8', 'B4', 'C5', 'F#2', '7.10.2', 'F4', 'B-4', '1.3.5.8', '0.1.4.8', '0.4', '0.1.4.8', '4.6.10', '7.9.0.2', '3.5.9.11', '11.3', '7.10.0.1.2', '7.10.2', '7.0', '7.10.0.2', '3.6.10', '4.6.10', '3.7.10', '5.8.0', '3.7.10', '9.11.3', '7.8.10', 'E-4', 'G#4', '8.9.2', '5.10', '7.10', '5.7.10.1', '10.1.5', 'C#2', '10.11.3', '1.5', '6.10.1', '0.4', '10.0.4', '8.10.0.4', 'F#6', '3.6', '10.1', '10.1.4', '6.8', '1.3', '8.10', '6.8', '10.1.3', '7.8', '5.6', '

- quelle est la taille de notre vocabulaire (le nombre de symbole différent que l'on devra encoder) ?

In [14]:
n_vocab = len(set(notes))
print(n_vocab)

1002


In [15]:
len(notes)

27869

### Comment effectuer une tache de génération ?

Le réseau prend en entrée une suite de note (par exemple, 100 notes), et doit prédire la 101eme.
Il faut donc :
- parser notre fichier de note en sequences de 100 notes (elles peuvent s'overlap)
- associer un label pour cette sequence qui est la 101eme note.
    - ex : une entree est constituee des notes 0 à 100 et son label la note 101. L'entrée suivante des notes 1 à 101, son label la note 102, etc...
- convertir les notes en format utilisable par le reseau de neurones. Quel est le format adapté pour représenter des notes ?
    - onehot ou ordinal ?

- via l'encoder adapté de scikit, encodons les notes :

In [36]:
notes_scikit = np.array(notes).reshape(-1, 1) 

# Entraine l'encoder. Pas beswoin de trier, car l'encoder scikit le fait pour nous !
enc = OrdinalEncoder()
enc.fit(notes_scikit)

# transform les notes originales
notes_encoded = enc.transform(notes_scikit)

In [41]:
print(notes[3], notes_encoded[3])

3.5.9 [418.]


- generer le jeu d'entrainement et les labels correspondant.
    - extraire des sequences de 100 notes comme entrées du reseau, et la note suivante comme label

In [48]:
network_input = []
network_output = []
sequence_size = 100
for i in range(len(notes_encoded)- sequence_size):
    network_input.append([val[0] for val in notes_encoded[i:i+sequence_size]])
    network_output.append(notes_encoded[i+sequence_size][0])

- normalisons l'entrée entre 0 et 1, et mettons tous ça au format attendu par keras

In [49]:
print(network_input[0])
print(network_output[0])

[956.0, 529.0, 956.0, 418.0, 73.0, 217.0, 741.0, 700.0, 422.0, 550.0, 441.0, 998.0, 926.0, 550.0, 926.0, 926.0, 926.0, 202.0, 202.0, 926.0, 992.0, 170.0, 946.0, 551.0, 953.0, 933.0, 969.0, 975.0, 773.0, 293.0, 778.0, 793.0, 962.0, 532.0, 441.0, 503.0, 139.0, 874.0, 876.0, 527.0, 685.0, 67.0, 73.0, 583.0, 82.0, 374.0, 780.0, 529.0, 74.0, 685.0, 183.0, 925.0, 777.0, 60.0, 529.0, 358.0, 776.0, 359.0, 926.0, 2.0, 214.0, 963.0, 689.0, 956.0, 155.0, 188.0, 582.0, 933.0, 947.0, 974.0, 700.0, 983.0, 926.0, 121.0, 21.0, 74.0, 21.0, 479.0, 743.0, 420.0, 286.0, 690.0, 700.0, 686.0, 693.0, 422.0, 479.0, 436.0, 583.0, 436.0, 911.0, 713.0, 962.0, 992.0, 848.0, 530.0, 688.0, 554.0, 186.0, 938.0]
208.0


In [56]:
n_patterns = len(network_input)
x_train = numpy.reshape(network_input, (n_patterns, 100, 1))/ float(n_vocab)
y_train = to_categorical(network_output)

In [57]:
print(x_train.shape)
print(y_train.shape)

(27769, 100, 1)
(27769, 1002)


## Creation du reseau et entrainement

- créer notre réseau recurrent
    - une ou plusieurs couches de lstm en entrée, suivit d'un perceptron pour prédire la note suivante

In [60]:
model = Sequential()


- entrainer le réseau : attention, cela peut prendre du temps !

In [61]:
model.fit(x_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
111/217 [==============>...............] - ETA: 4:19 - loss: 6.0305

KeyboardInterrupt: 

### Prediction

- on tire une sequence au hasard pour initialiser notre prediction
    - on aurait aussi pu lui proposer une ou plusieurs notes aléatoires
- on le laisse compléter note par note

In [66]:
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
current_input = network_input[start]
prediction_output = []

n = 250
for note_index in range(n):
    prediction_input = numpy.reshape(current_input, (1, len(current_input), 1))
    prediction_input = prediction_input / float(n_vocab)

    # effectue la prediction de la  note suivante
    prediction = model.predict(prediction_input, verbose=0)
    pred = numpy.argmax(prediction)
    
    # converti la note prédite au format mid
    pred_note = enc.inverse_transform([[pred]])[0]
    prediction_output.append(pred_note[0])

    # ajoute la note prédite à la fin de l'entrée, et décale celle-ci d'une note
    current_input.append(pred)
    current_input = current_input[1:]

- on peut convertir la sortie au format midi pour l'écouter

In [67]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='output.mid')

create_midi(prediction_output)

In [68]:
print(prediction_output)

['F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2', 'F#2'

## Ca ne marche pas ? 
- augmentez la capacité du reseau
    - dans la litterature, jusqu'à 3 couches de lstm
- entrainez plus longtemps
    - dans la litterature, 200 epoques
    - vous devrez probablement le laisser tourner la nuit pour obtenir un résultat
- vous pouvez écouter des résultats de réseaux similaire 
    - https://www.rileynwong.com/blog/2019/2/25/generating-music-with-an-lstm-neural-network
    - https://becominghuman.ai/generating-music-using-lstm-neural-network-545f3ac57552